In [1]:
import re
from pyfaidx import Fasta
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pyarrow.dataset as ds

from dataset import GeneIterableFixedB, split_filters_by_chroms
import torch
import pprint

/home/eddiekong/ml4fg/isoform-gen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
usecols = [0, 2, 3, 4, 6, 8]
colnames = ["chrom", "feature", "start", "end", "strand", "attributes"]

gtf = pd.read_csv(
    "data/raw_data/ENCFF129ZDE.gtf.gz",
    sep="\t",
    comment="#",
    header=None,
    names=colnames,
    usecols=usecols,
    compression="gzip"
)

In [8]:
# row = gtf.iloc[1]
# import pprint
# pprint.pprint(row.to_dict(), width=120)

gtf.iloc[12:30]

,chrom,feature,start,end,strand,attributes
12,chr1,gene,14404,29321,-,"gene_id ""ENSG00000227232.5""; gene_name ""WASH7P..."
13,chr1,transcript,14404,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
14,chr1,exon,29534,29570,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
15,chr1,exon,24738,24891,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
16,chr1,exon,18268,18366,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
17,chr1,exon,17915,18061,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
18,chr1,exon,17606,17742,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
19,chr1,exon,17233,17368,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
20,chr1,exon,16858,17055,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."
21,chr1,exon,16607,16765,-,"gene_id ""ENSG00000227232.5""; transcript_id ""EN..."


In [30]:
# read the Parquet file
df = pd.read_parquet("data/table/ENCFF180NXA.parquet")

# number of rows
print(len(df))

129911


In [31]:
row = df.iloc[18197]
pprint.pprint(row.to_dict(), width=120)

num_transcripts = df["transcripts"].apply(len).sum()

print("Transcripts:", num_transcripts)

{'chrom': 'chr16',
 'end': 57591681,
 'gene_id': 'ENSG00000159618.15',
 'start': 57542421,
 'strand': '+',
 'transcripts': array([{'transcript_id': 'ENST00000615867.4', 'tx_start': 57542687, 'tx_end': 57565313, 'exon_starts': array([57542687, 57562056, 57562384, 57563091, 57563848, 57565034]), 'exon_ends': array([57542701, 57562157, 57562459, 57563247, 57563979, 57565313]), 'rel_abundance': 1.0}],
      dtype=object)}
Transcripts: 351270


In [36]:
target = "ENCLB376UFGT000206877"

matches = df[df["transcripts"].apply(
    lambda txs: any(t["transcript_id"] == target for t in txs)
)]

print("Number of matching genes:", len(matches))
matches[["gene_id", "chrom", "start", "end"]]

Number of matching genes: 0


,gene_id,chrom,start,end
